In [78]:
#system libraries
import os
import sys
import random
import time
#selenium libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.chrome.options import Options
#recaptcha libraries
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub

In [52]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [109]:
try:
    #create chrome driver
    driver = webdriver.Chrome('chromedriver')
    #go to website
    driver.get("https://www.gob.mx/curp/")
    time.sleep(random.randint(2,3))
except:
    print("[-] Please update the chromedriver.exe")

driver.find_element("xpath",'//*[@id="curpinput"]').send_keys('OICL980815HDFLNN01')

while True:
    try:
        #switch to recaptcha frame
        frames=driver.find_element("xpath", '//*[@id="ember313"]/div/div/iframe')
        driver.switch_to.frame(frames);

        #click on checkbox to activate recaptcha
        driver.find_element("xpath",'//*[@id="recaptcha-anchor"]/div[1]').click()
        break
    except NoSuchElementException:
        pass
time.sleep(random.randint(1,2))

validar_captcha = driver.find_element("xpath",'//*[@id="recaptcha-anchor"]/div[1]').get_attribute('style')

if not validar_captcha == 'display: none;':
    while True:
        try:
            #switch to recaptcha audio control frame
            driver.switch_to.default_content()
            frames=driver.find_element("xpath",'/html/body/div[3]/div[4]/iframe')
            driver.switch_to.frame(frames)

            #click on audio challenge
            driver.find_element("xpath",'//*[@id="recaptcha-audio-button"]').click()
            break
        except NoSuchElementException:
            pass
    time.sleep(random.randint(1,2))

    while True:
        try:
            #switch to recaptcha audio challenge frame
            driver.switch_to.default_content()
            frames= driver.find_element("xpath","/html/body/div[3]/div[4]/iframe")
            driver.switch_to.frame(frames)

            #get the mp3 audio file
            src = driver.find_element("xpath",'//*[@id="audio-source"]').get_attribute("src")
            break
        except NoSuchElementException:
            pass
    time.sleep(random.randint(1,2))
    
    while True:
        #download the mp3 audio file from the source
        urllib.request.urlretrieve(src, "sample.mp3")
        #convert mp3 to wav
        sound = pydub.AudioSegment.from_mp3("./sample.mp3")
        sound.export("sample.wav", format="wav")
        sample_audio = sr.AudioFile("./sample.wav")
        r = sr.Recognizer()
        with sample_audio as source:
            audio = r.record(source)

        #translate audio to text with google voice recognition
        key=r.recognize_google(audio)

        #key in results and submit
        driver.find_element("xpath",'//*[@id="audio-response"]').send_keys(key.lower())
        driver.find_element("xpath",'//*[@id="audio-response"]').send_keys(Keys.ENTER)
        try:
            error = driver.find_element("xpath",'/html/body/div/div/div[1]')
            if not error.text == 'Multiple correct solutions required - please solve more.':
                break
            src = driver.find_element("xpath",'//*[@id="audio-source"]').get_attribute("src")
        except NoSuchElementException:
            break

driver.switch_to.default_content()
driver.find_element("xpath",'//*[@id="searchButton"]').click()
time.sleep(random.randint(1,2))

try:
    driver.find_element("xpath",'//*[@id="modalMessage"]')
    print('CURP no valido.')
except NoSuchElementException:
    table = driver.find_elements("xpath",'//*[@id="ember336"]/section/div[1]/div/div[2]/form/div[2]/div[1]/div/div[2]/table/tr')
    resultados = []
    for index in range(len(table)):
        values = table[index].find_elements("tag name", "td")
        resultados.append(values[1].text)
    print(resultados)

['OICL980815HDFLNN01', 'LEONEL', 'OLIVARES', 'CONCHILLOS', 'HOMBRE', '15/08/1998', 'MEXICO', 'DISTRITO FEDERAL', 'ACTA DE NACIMIENTO']
